In [ ]:
import cupy as cp
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from numba.core.errors import NumbaPerformanceWarning
import numpy as np
import os
import pickle
import sys
import warnings
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)
os.environ['CUPY_ACCELERATORS'] = 'cub,cutensor'

sys.path.append('..')
from src import *

# Use `ipympl` for VS Code and `inline` for Jupyter/Jupyterlab
%matplotlib ipympl

The lattice constant for the triangular lattice potential we use is much larger than that of the graphene lattice, so we first need to determine the ideal box width that will simultaneously satisfy periodic boundary conditions for both. The `ideal_boxwidth()` function compares the two lattices and various box sizes up to the `target_width` and returns the box widths in the order of the smallest to the largest error. For the best simulation accuracy, it is ideal to use a box width that is zero or as close to zero as possible.

In [ ]:
ideal_boxwidths(tri_lc=3.936, kag_lc=0.246, target_width=300, show_topn=5) 

With this we can choose the boxwidth with a low comparison error that is large enough for our needs. The left subplot below shows the molecules overlayed on top of the triangular lattice energy surface. The right subplot shows the Voronoi plot of each molecule with its nearest-neighboring molecules as a number of disclinations, which is one of many potentially useful indicators of phase.

In [ ]:
boxwidth    = 102.336  # Simulation box width in nanometers
n_molecules = 800      # Number of molecules to generate #400, 800, 1200, 1600, 2000
threshold   = 10       # Minimum distance (in lattice constants) between initial molecule positions #17, 12, 9, 8, 7

# Define the Kagome lattice object for the simulation (all its parameters are shown here)
# Note that the data types defined here are used for the entire simulation
kagome = KagomeLattice(
    boxwidth=boxwidth, 
    lattice_constant=0.246, 
    energy_barrier=275.0, 
    debye_freq=10e10, 
    transition_type='t', 
    intdtype='int32', 
    floatdtype='float64'
)

# The ".get_latticesites()" method to get the array of all lattice sites xy-coordinates
# An array of position indices can also be passed to get the xy-coordinates of specific lattice sites
lxy = kagome.get_latticesites(site_numbers=None)

# The ".get_sitennids()" method retrieves the array of all nearest neighbor indices for each lattice site
lnnids = kagome.get_sitennids(site_numbers=cp.arange(lxy.shape[0], dtype=kagome._idtype))

# The ".generate_randomids()" method generates random molecule positions on the lattice
if n_molecules > 0: 
    mids = kagome.generate_randomids(
        n_molecules=n_molecules, 
        sites=lxy, 
        n_attempts=1000, 
        threshold=threshold
    )

# The TriangularLatticeEnergies object is used to compute the triangular lattice energy surface
# The ".U(x,y)" method computes the energy at a given (x,y) position or array of positions
tle = TriangularLatticeEnergies(lattice_constant=3.936, amplitude=39.8, angle=0., shift=[0., 0.])

# Compute packing fraction density (for comparison, experiment e8_217.csv was: 0.037308)
xdim, ydim = kagome.boxsize
rho = (n_molecules*np.pi*0.48*0.87) / (2*xdim*ydim)
print(f"System box dimensions (nm): ({kagome.boxsize[0]:.4f}, {kagome.boxsize[1]:.4f})")
print(f"Number of Kagome unit cells (columns, rows): {kagome.n_unitcells}")
print(f"Number of total Kagome lattice sites: {kagome._n_totalsites}")
print(f"Packing fraction density: {rho}")

# The "plotter.py" module contains some functions to plot the triangular lattice energies, lattice sites, bonds, and molecules
fig, axs = plt.subplots(1, 2, figsize=(25, 11*figsize_yscale(kagome)), layout='constrained', dpi=72)
fig.canvas.resizable = False
# Plot the lattice energy surface
axs[0] = plot_latticeenergies(axs[0], tle, kagome.boxsize, nsamples=1000, cmap='viridis', alpha=0.75, zorder=1)
if boxwidth <= 10:
    # Plot the graphene lattice bonds
    axs[0] = plot_graphenebonds(axs[0], kagome, colors='b', lw=0.5, ls='--', alpha=0.25, zorder=2)
    # Plot the Kagome lattice sites
    axs[0] = plot_kagomesites(axs[0], kagome, s=18, c='k', ec='k', alpha=0.3, zorder=3)
if n_molecules > 0:
    # Plot the molecules
    axs[0] = plot_molecules(axs[0], lxy[lnnids[mids]].get(), show_nn=False, s=35, lw=1.0, mc='tab:red')
# Plot the hexagonal order parameter
fig, axs[1] = plot_disclinations(fig, axs[1], kagome, lxy[mids].get())
for i in range(2):
    axs[i].set(xlim=kagome.xlim, ylim=kagome.ylim)
    axs[i].set_xlabel('x (nm)', fontsize=14)
    axs[i].set_ylabel('y (nm)', fontsize=14)
plt.show()

### Simulation

Below is the code to start an actual simulation run. The previous lattice object `kagome` is passed to the `simulate()` function, in addition to the previously defined `n_molecules` and `threshold` parameters.

In [ ]:
# Save path for the pickle file containing the simulation data
save_path = '../data/simdata.pkl'

duration          = 0.001   # Duration of the simulation in seconds
frames_per_kelvin = 30      # Number of frames to sample per temperature change in Kelvin
nwarmupsteps      = 100000  # Number of warm-up steps to take before the simulation starts
progress_freq     = 250000  # Frequency of the progress output in KMC steps
max_dt            = 60      # Maximum time step interval threshold in seconds

# Energy parameters for the simulation
energy_params = {
    'lattice':'none',  # Lattice energy can either be 'triangular' or 'none'
    'lattice_params':{
        'lattice_constant':3.936,  # Lattice constant of the triangular lattice (nm)
        'amplitude':39.8,          # Amplitude of the triangular lattice energy (meV)
        'angle':0.0,               # Angle of the triangular lattice (degrees)
        'shift':[0.0, 0.0]         # Shift of the triangular lattice (nm)
    },
    'interaction':'total_impurity',  # Interaction energy parameters
    'interaction_params':{
        'Zval':1.0,  # Impurity charge value
        'epsr':4.22  # Dielectric constant of the medium 
    }
}

# Temperature parameters for the simulation
temperature_params = {
    'temp_initial':30,  # Initial temperature of the simulation (K)
    'temp_final':8.5,   # Final temperature of the simulation (K)
    'method':'linear'   # Cooling method can be 'linear', 'exponential', or 'inv_exponential'
}

# Molecule parameters for the simulation
molecule_params = {
    'n_molecules':n_molecules,  # Number of molecules to generate
    'threshold':threshold,      # Minimum distance (in lattice constants) between initial molecule positions
}

In [ ]:
# Uncomment this cell to run a new simulation and save the data
# The `simulate()` function is used to run the simulation and return the simulation data as a dictionary
simdata = simulate(kagome, energy_params, temperature_params, molecule_params, duration, frames_per_kelvin=frames_per_kelvin, nwarmupsteps=nwarmupsteps, progress_freq=progress_freq, max_dt=max_dt)

# Save the simulation data
with open(save_path, 'wb') as f:
    pickle.dump(simdata, f)

### Load saved data

In [ ]:
# # Uncomment this cell to load previous simulation data
# with open(save_path, 'rb') as f:
#     simdata = pickle.load(f)

We can see all of the keys contained in `simdata` and an example of some of the data. We can also see that this output also contains all the parameters you need to recreate the simulation if you wanted to.

In [ ]:
print(simdata.keys())
print(f'\ntimes (in seconds): {simdata["times"].shape}\n{simdata["times"][-10:]}\n')
print(f'temperatures (in Kelvin): {simdata["temperatures"].shape}\n{simdata["temperatures"][-10:]}\n')
print(f'ids: {simdata["ids"].shape}\n{simdata["ids"][0,-10:]}')

We can also visualize the simulation and save the animation to a `.gif` file.

In [ ]:
ani = animate_simulation(simdata, save_filepath=save_path[:-4]+'.gif', frameskip=5, interval=33, repeat=True, figsize=(28, 10))  #save_path[:-4]+'.gif'